<a href="https://colab.research.google.com/github/pawel-wyszomirski/metadas_analiza_wieku/blob/main/Meta_Ads_analiza_wieku_(sprzeda%C5%BC%2C_roas%2C_wy%C5%9Bwietlenia).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from google.colab import files
import ipywidgets as widgets
from IPython.display import clear_output

def create_plots(data):
    """Tworzy i wyświetla wykresy"""
    # Utworzenie 4 wykresów w układzie 2x2
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 12))

    # Wykres 1: Liczba sprzedaży
    data['Liczba sprzedaży'].plot(kind='bar', ax=ax1, color='skyblue')
    ax1.set_title('Liczba sprzedaży w grupach wiekowych')
    ax1.set_xlabel('Grupa wiekowa')
    ax1.set_ylabel('Liczba sprzedaży')
    ax1.tick_params(axis='x', rotation=45)

    # Wykres 2: ROAS
    data['ROAS'].plot(kind='bar', ax=ax2, color='lightgreen')
    ax2.set_title('ROAS w grupach wiekowych')
    ax2.set_xlabel('Grupa wiekowa')
    ax2.set_ylabel('ROAS')
    ax2.tick_params(axis='x', rotation=45)

    # Wykres 3: Przychody i wydatki
    x = range(len(data.index))
    width = 0.35
    ax3.bar(x, data['Przychód (PLN)'], width, label='Przychód (PLN)', color='lightgreen')
    ax3.bar([i + width for i in x], data['Wydatki (PLN)'], width, label='Wydatki (PLN)', color='lightcoral')
    ax3.set_title('Przychody i wydatki w grupach wiekowych')
    ax3.set_xlabel('Grupa wiekowa')
    ax3.set_ylabel('PLN')
    ax3.set_xticks([i + width/2 for i in x])
    ax3.set_xticklabels(data.index, rotation=45)
    ax3.legend()

    # Wykres 4: Liczba wyświetleń
    data['Wyświetlenia'].plot(kind='bar', ax=ax4, color='orange')
    ax4.set_title('Liczba wyświetleń w grupach wiekowych')
    ax4.set_xlabel('Grupa wiekowa')
    ax4.set_ylabel('Wyświetlenia')
    ax4.tick_params(axis='x', rotation=45)

    # Dopasowanie układu
    plt.tight_layout()
    plt.show()

def analyze_with_aov(aov_value):
    """Analizuje dane z określonym AOV"""
    print(f"Używam AOV = {aov_value} PLN")
    print("\nTeraz wybierz plik CSV z danymi kampanii...")

    uploaded = files.upload()
    if not uploaded:
        print("Nie wybrano pliku!")
        return

    file_name = next(iter(uploaded))
    print(f"Wczytano plik: {file_name}")

    try:
        # Wczytanie danych
        encodings = ['utf-8', 'latin1', 'cp1250']
        for encoding in encodings:
            try:
                df = pd.read_csv(file_name, encoding=encoding)
                break
            except UnicodeDecodeError:
                continue

        # Podstawowe obliczenia
        df['Wyniki'] = df['Wyniki'].fillna(0)
        df['Wyniki'] = df['Wyniki'].astype(int)  # Konwersja na liczby całkowite
        df['Przychód'] = df['Wyniki'] * aov_value

        # Grupowanie według wieku
        grouped = df.groupby('Wiek').agg({
            'Nazwa reklamy': 'count',
            'Wydana kwota (PLN)': 'sum',
            'Wyświetlenia': 'sum',
            'Kliknięcia (wszystkie)': 'sum',
            'Wyniki': 'sum'
        })

        # Konwersja wyświetleń i kliknięć na int
        grouped['Wyświetlenia'] = grouped['Wyświetlenia'].astype(int)
        grouped['Kliknięcia (wszystkie)'] = grouped['Kliknięcia (wszystkie)'].astype(int)

        # Dodatkowe obliczenia
        grouped['Przychód (PLN)'] = grouped['Wyniki'] * aov_value
        grouped['ROAS'] = (grouped['Przychód (PLN)'] / grouped['Wydana kwota (PLN)']).round(2)
        grouped['Koszt pozyskania sprzedaży'] = (grouped['Wydana kwota (PLN)'] / grouped['Wyniki'].replace(0, float('inf'))).round(2)

        # Zmiana nazw kolumn
        grouped.columns = [
            'Liczba reklam',
            'Wydatki (PLN)',
            'Wyświetlenia',
            'Kliknięcia',
            'Liczba sprzedaży',
            'Przychód (PLN)',
            'ROAS',
            'Koszt pozyskania sprzedaży'
        ]

        # Podsumowanie
        print("\nPODSUMOWANIE KAMPANII REVOLUT\n")
        print(f"Całkowita liczba sprzedaży: {int(grouped['Liczba sprzedaży'].sum())}")
        print(f"Całkowite wydatki: {grouped['Wydatki (PLN)'].sum():.2f} PLN")
        print(f"Całkowity przychód: {grouped['Przychód (PLN)'].sum():.2f} PLN")
        print(f"Średni ROAS: {(grouped['Przychód (PLN)'].sum() / grouped['Wydatki (PLN)'].sum()):.2f}")

        # Szczegółowa tabela
        print("\nSzczegółowe statystyki dla grup wiekowych:")
        display(grouped.style.format({
            'Liczba reklam': '{:.0f}',
            'Wydatki (PLN)': '{:.2f}',
            'Wyświetlenia': '{:.0f}',
            'Kliknięcia': '{:.0f}',
            'Liczba sprzedaży': '{:.0f}',
            'Przychód (PLN)': '{:.2f}',
            'ROAS': '{:.2f}',
            'Koszt pozyskania sprzedaży': '{:.2f}'
        }))

        # Generowanie wykresów
        print("\nGeneruję wykresy...")
        create_plots(grouped)

        # Zapisz do Excela
        excel_name = 'raport_kampanii_revolut.xlsx'
        grouped.to_excel(excel_name)
        print(f"\nZapisano szczegółowy raport do pliku '{excel_name}'")
        files.download(excel_name)

    except Exception as e:
        print(f"Wystąpił błąd: {str(e)}")
        import traceback
        print(traceback.format_exc())
        print("\nUpewnij się, że:")
        print("1. Plik jest w formacie CSV")
        print("2. Plik zawiera wszystkie wymagane kolumny")
        print("3. Dane są poprawnie sformatowane")

# Widget do wprowadzania AOV
aov_input = widgets.FloatText(
    value=17.0,
    description='AOV (PLN):',
    disabled=False
)

button = widgets.Button(description='Rozpocznij analizę')
output = widgets.Output()

def on_button_clicked(b):
    with output:
        clear_output()
        if aov_input.value <= 0:
            print("AOV musi być większe od 0!")
            return
        analyze_with_aov(aov_input.value)

button.on_click(on_button_clicked)

print("Podaj średnią wartość zamówienia (AOV):")
display(aov_input)
display(button)
display(output)

Podaj średnią wartość zamówienia (AOV):


FloatText(value=17.0, description='AOV (PLN):')

Button(description='Rozpocznij analizę', style=ButtonStyle())

Output()